In [52]:
import sys
sys.path.insert(0, '/Users/niyi/Documents/GitHub/Optimal-Control/Tools')
from ActorCriticNetworks import ActorNetwork, CriticNetwork
import torch as T
import torch.nn as nn 

In [53]:
N= CriticNetwork(5, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='tb', learning_rate=0.02)

bt= T.tensor([[-1.4355, -0.7806,  0.3042,  1.1601, -0.1184]])
at= T.tensor([[0.8233, 0.8126]])
# for target_param in N.parameters():
#     print(target_param)
bt, at, N(bt, at)

(tensor([[-1.4355, -0.7806,  0.3042,  1.1601, -0.1184]]),
 tensor([[0.8233, 0.8126]]),
 tensor([[-0.2737]], grad_fn=<AddmmBackward0>))

In [54]:
Nt= ActorNetwork(5, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='bt', learning_rate=0.02)

btt= T.tensor([[-0.6552,  0.0852,  2.0087, -0.6352,  0.4445]])
att= T.randn(1, 2)

# Nt.load_checkpoint()
btt, Nt(btt)

TypeError: ActorNetwork.__init__() got multiple values for argument 'name'